In [ ]:
import numpy as np
import nltk
import string
import random
import re
!pip install chatterbot
!pip install chatterbot_corpus
from chatterbot import ChatBot
from chatterbot.trainers import ChatterBotCorpusTrainer
import pathlib

In [ ]:
botname = 'Norman'
chatbot = ChatBot(botname)

chatbot = ChatBot(botname, 
	#storage_adapter='chatterbot.storage.SQLStorageAdapter',
	logic_adapters=[
        {
            'import_path': 'chatterbot.logic.BestMatch',
            'default_response': 'None',
            'maximum_similarity_threshold': 0.8 # tune this threshold
        },
        {
            "import_path": "chatterbot.logic.MathematicalEvaluation",

        },
        # {
        #     "import_path": "chatterbot.logic.UnitConversion",

        # },
        # {
        #     "import_path": "profanity_adapter.ProfanityAdapter",

        # },
        # {
        #     "import_path": "covid19_adapter.Covid19Adapter",

        # },
    ],
 )

In [ ]:
# Training list data for question answers properly
# from chatterbot.trainers import ListTrainer

# trainer = ListTrainer(bot)

# trainer.train([
#     'How are you?',
#     'I am good.',
#     'That is good to hear.',
#     'Thank you',
#     'You are welcome.',
# ]

In [ ]:
# trainer = ChatterBotCorpusTrainer(chatbot)

# trainer.train(
#     str(pathlib.Path().absolute())+'/english/',
# )

trainer = ChatterBotCorpusTrainer(chatbot)

trainer.train(
    "chatterbot.corpus.english"
)

Training ai.yml: [####################] 100%
Training botprofile.yml: [####################] 100%
Training computers.yml: [####################] 100%
Training conversations.yml: [####################] 100%
Training emotion.yml: [####################] 100%
Training food.yml: [####################] 100%
Training gossip.yml: [####################] 100%
Training greetings.yml: [####################] 100%
Training health.yml: [####################] 100%
Training history.yml: [####################] 100%
Training humor.yml: [####################] 100%
Training literature.yml: [####################] 100%
Training money.yml: [####################] 100%
Training movies.yml: [####################] 100%
Training politics.yml: [####################] 100%
Training psychology.yml: [####################] 100%
Training science.yml: [####################] 100%
Training sports.yml: [####################] 100%
Training trivia.yml: [####################] 100%


In [ ]:
f = open('corpus.txt', errors='strict')
data = f.read()
data = data.lower()
nltk.download('punkt')
nltk.download('wordnet')
sent_tokens = nltk.sent_tokenize(data)
word_tokens = nltk.word_tokenize(data)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
sent_tokens[:3]

['at coding ninjas, our mission is to continuously innovate the best ways to train the next generation of developers and to transform the the way tech education is delivered.',
 'coding ninjas was founded in 2016 to bridge the knowledge gap between colleges and industry.',
 'founded by ankush singla, kannu mittal and dhawal parate, coding ninjas boasts of world-class teaching faculty and a state-of-art learning platform for coding education with faculty alumni of iit, stanford, iiit and facebook.']

In [ ]:
word_tokens[:3]

['at', 'coding', 'ninjas']

In [ ]:
lemmer = nltk.stem.WordNetLemmatizer()
def LemTokens(tokens):
  return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict= dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
  return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

### Response Generation

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def refer_chat(user_input):
  robo1_response = ''
  TfidfVec = TfidfVectorizer(tokenizer = LemNormalize, stop_words='english')
  tfidf = TfidfVec.fit_transform(sent_tokens)
  vals = cosine_similarity(tfidf[-1], tfidf)
  idx = vals.argsort()[0][-2]
  flat = vals.flatten()
  flat.sort()
  req_tfidf = flat[-2]
  if (req_tfidf == 0):
    robo1_response= robo1_response+"I could not answer this right now but you can contact the head of our dept." # add the dept recommendation engine and contact details
    return robo1_response
  else:
    robo1_response = robo1_response+sent_tokens[idx]
    return robo1_response

## Defining start and end of chat

In [ ]:
greet_input = ('hi', 'hello', 'hey', 'sup', 'whats up', 'hai')
greet_response = ('hello', 'nods', 'welcome', 'thanks for chosing me', 'glab atleast you are talking to me')
def greet(user_input):
  for word in user_input.split():
    if word.lower() in greet_input:
      return random.choice(greet_response)

In [ ]:
def message_probability(user_message, recognised_words, single_response=False, required_words=[]):
    message_certainty = 0
    has_required_words = True

    # Counts how many words are present in each predefined message
    for word in user_message:
        if word in recognised_words:
            message_certainty += 1

    # Calculates the percent of recognised words in a user message
    percentage = float(message_certainty) / float(len(recognised_words))

    # Checks that the required words are in the string
    for word in required_words:
        if word not in user_message:
            has_required_words = False
            break

    # Must either have the required words, or be a single response
    if has_required_words or single_response:
        return int(percentage * 100)
    else:
        return 0

In [ ]:
def check_all_messages(message):
    highest_prob_list = {}

    # Simplifies response creation / adds it to the dict
    def response(bot_response, list_of_words, single_response=False, required_words=[]):
        nonlocal highest_prob_list
        highest_prob_list[bot_response] = message_probability(message, list_of_words, single_response, required_words)

    # Responses -------------------------------------------------------------------------------------------------------
    response('Hello!', ['hello', 'hi', 'hey', 'sup', 'heyo'], single_response=True)
    response('See you!', ['bye', 'goodbye'], single_response=True)
    response('I\'m doing fine, and you?', ['how', 'are', 'you', 'doing'], required_words=['how'])
    response('ask more!', ['good', 'am', 'I', 'fine', 'nice', 'ok', 'okay', 'kk'], single_response=True)
    response('Me too! then?', ['i', 'love', 'coding', 'ninjas', 'lpu'], required_words=['love'])

    best_match = max(highest_prob_list, key=highest_prob_list.get)
    # print(highest_prob_list)
    # print(f'Best match = {best_match} | Score: {highest_prob_list[best_match]}')

    return None if highest_prob_list[best_match] < 1 else best_match


In [ ]:
flag = True
print('BOT: My name is ninja_assitant, Lets have a conversation, I am hear to help you with any question, doubt, details etc. Anytime, if you would like to quit just say bye!!')
while (flag):
  user_input = input()
  user_input = user_input.lower()# re.split(r'\s+|[,;?!.-]\s*', user_input.lower())
  if (user_input != 'bye'):
    if (user_input == 'Thanks' or user_input =='Thank you' or user_input == 'Tnx' or user_input =='Thankq' or user_input == 'Tq'):
      flag = False
      print('BOT: Thank you, nice talking to you')
    else:
      if (greet(user_input)!=None):
        print('BOT: '+greet(user_input))
      elif check_all_messages(user_input.split()) !=None:
        print('BOT: '+check_all_messages(user_input.split()))
      elif str(chatbot.get_response(user_input))!= 'None':
        print('BOT: '+str(chatbot.get_response(user_input)))
      else:
        sent_tokens.append(user_input)
        word_tokens = word_tokens + nltk.word_tokenize(user_input)
        final = list(set(word_tokens))
        print('BOT: ', end='')
        print(refer_chat(user_input))
        sent_tokens.remove(user_input)
  else:
    flag = False
    print('BOT: Good bye, visit again')

BOT: My name is ninja_assitant, Lets have a conversation, I am hear to help you with any question, doubt, details etc. Anytime, if you would like to quit just say bye!!
hi
BOT: glab atleast you are talking to me
how r u
BOT: I'm doing fine, and you?
founded year


/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  % sorted(inconsistent)


BOT: coding ninjas was founded in 2016 to bridge the knowledge gap between colleges and industry.
alumni


/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  % sorted(inconsistent)


BOT: today, coding ninjas ecosystem comprises of 40,000+ students and alumni, 1000+ campus ambassadors, 2000+ teaching assistants, and 150+ employees.
what is spiderman
BOT: a comic book story made into a movie.
bye
BOT: Good bye, visit again


### To-do
1. options of topics
2. statics response types
3. nltk response types
4. data for various sections and use grammerly
5. contact dept head or our team.
6. interactive sentances to make a feel of bot as amazing as alexa or siri by having sentances like "I was an avg student to answer so leave a message to the cleverest of our team i.e., name in random again for dept heads"
7. how to integrate bot and website ? some one can join me who has experience on stuff like this i.e., we get widget for free and input on website is taken on to this python file and then use that to generate and display response of the bot to chat screen.

Help:
1. data collection help on diff sections with grammerly.
2. a skilled one to help me integrate in some random web page and integrate to whole website after testing.